In [1]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark import SparkConf
from dotenv import load_dotenv
import os
from datetime import datetime
import pandas as pd 

In [2]:
query_counts = """
SELECT
    YEAR,
    MONTH,
    SERVICE_TYPE,
    COUNT(*) AS total_viajes,
    ROUND(AVG(TRIP_DISTANCE), 2) AS avg_distance,
    ROUND(MAX(TRIP_DISTANCE), 2) AS max_distance,
    ROUND(MIN(TRIP_DISTANCE), 2) AS min_distance,
    ROUND(AVG(TOTAL_AMOUNT), 2) AS avg_total_amount,
    ROUND(MAX(TOTAL_AMOUNT), 2) AS max_total_amount,
    ROUND(MIN(TOTAL_AMOUNT), 2) AS min_total_amount
FROM STAGE.OBT_TRIPS
GROUP BY YEAR, MONTH, SERVICE_TYPE
ORDER BY YEAR, MONTH, SERVICE_TYPE;
"""

In [3]:
import snowflake.connector

conn = snowflake.connector.connect(
    #Se tiene que cambiar a los secretosAAAAAAA
    user="MARE122510",
    password="MyTurnEra2025100%",
    account="TLZAPUN-PKC06603",
    warehouse="COMPUTE_WH",
    database="NY_TAXI",
    schema="STAGE",
    role =os.getenv('ROLE')
)

cur = conn.cursor()
cur.execute(query_counts)
print("cursor creado")
results = cur.fetchall()

# Obtener nombres de columnas
columns = [desc[0] for desc in cur.description]

# Mostrar como tabla
for row in results:
    print(dict(zip(columns, row)))

cur.close()

cursor creado
{'YEAR': 2000, 'MONTH': 12, 'SERVICE_TYPE': 'yellow', 'TOTAL_VIAJES': 18, 'AVG_DISTANCE': 5.55, 'MAX_DISTANCE': 20.77, 'MIN_DISTANCE': 0.11, 'AVG_TOTAL_AMOUNT': 27.62, 'MAX_TOTAL_AMOUNT': 98.76, 'MIN_TOTAL_AMOUNT': 3.8}
{'YEAR': 2001, 'MONTH': 1, 'SERVICE_TYPE': 'yellow', 'TOTAL_VIAJES': 4, 'AVG_DISTANCE': 3.83, 'MAX_DISTANCE': 9.13, 'MIN_DISTANCE': 0.0, 'AVG_TOTAL_AMOUNT': 21.85, 'MAX_TOTAL_AMOUNT': 43.0, 'MIN_TOTAL_AMOUNT': 3.3}
{'YEAR': 2001, 'MONTH': 2, 'SERVICE_TYPE': 'yellow', 'TOTAL_VIAJES': 1, 'AVG_DISTANCE': 0.0, 'MAX_DISTANCE': 0.0, 'MIN_DISTANCE': 0.0, 'AVG_TOTAL_AMOUNT': 3.8, 'MAX_TOTAL_AMOUNT': 3.8, 'MIN_TOTAL_AMOUNT': 3.8}
{'YEAR': 2001, 'MONTH': 8, 'SERVICE_TYPE': 'yellow', 'TOTAL_VIAJES': 1, 'AVG_DISTANCE': 7.08, 'MAX_DISTANCE': 7.08, 'MIN_DISTANCE': 7.08, 'AVG_TOTAL_AMOUNT': 24.55, 'MAX_TOTAL_AMOUNT': 24.55, 'MIN_TOTAL_AMOUNT': 24.55}
{'YEAR': 2002, 'MONTH': 2, 'SERVICE_TYPE': 'yellow', 'TOTAL_VIAJES': 11, 'AVG_DISTANCE': 2.93, 'MAX_DISTANCE': 11.28, 'MIN

True

In [4]:
query_val = """
WITH base AS (
    SELECT *
    FROM STAGE.OBT_TRIPS
)
SELECT
    -- TOTAL DE REGISTROS
    COUNT(*) AS total_registros,
    
    -- NULOS
    COUNT_IF(PICKUP_DATETIME IS NULL) AS null_pickup_datetime,
    COUNT_IF(DROPOFF_DATETIME IS NULL) AS null_dropoff_datetime,
    COUNT_IF(PICKUP_DATE IS NULL) AS null_pickup_date,
    COUNT_IF(PU_LOCATION_ID IS NULL) AS null_pu_location,
    COUNT_IF(DO_LOCATION_ID IS NULL) AS null_do_location,
    COUNT_IF(TRIP_DISTANCE IS NULL) AS null_trip_distance,
    COUNT_IF(TOTAL_AMOUNT IS NULL) AS null_total_amount,
    COUNT_IF(SERVICE_TYPE IS NULL) AS null_service_type,
    COUNT_IF(PAYMENT_TYPE_DESC IS NULL) AS null_payment_type,

    -- RANGOS
    COUNT_IF(TRIP_DISTANCE < 0 OR TRIP_DISTANCE > 200) AS distancia_fuera_rango,
    COUNT_IF(TOTAL_AMOUNT < 0 OR TOTAL_AMOUNT > 2000) AS monto_fuera_rango,
    COUNT_IF(PASSENGER_COUNT < 0 OR PASSENGER_COUNT > 10) AS pasajeros_fuera_rango,
    COUNT_IF(AVG_SPEED_MPH < 0 OR AVG_SPEED_MPH > 120) AS velocidad_fuera_rango,
    COUNT_IF(TIP_PCT < 0 OR TIP_PCT > 100) AS tip_pct_fuera_rango,
    -- COHERENCIA DE FECHAS
    COUNT_IF(DROPOFF_DATETIME < PICKUP_DATETIME) AS dropoff_antes_pickup,
    COUNT_IF(TRIP_DURATION_MIN <= 0) AS duracion_invalida,

    -- ESTADÍSTICAS
    MIN(TRIP_DISTANCE) AS min_trip_distance,
    MAX(TRIP_DISTANCE) AS max_trip_distance,
    AVG(TRIP_DISTANCE) AS avg_trip_distance,
    
    MIN(TOTAL_AMOUNT) AS min_total_amount,
    MAX(TOTAL_AMOUNT) AS max_total_amount,
    AVG(TOTAL_AMOUNT) AS avg_total_amount,
    
    MIN(TRIP_DURATION_MIN) AS min_trip_duration,
    MAX(TRIP_DURATION_MIN) AS max_trip_duration,
    AVG(TRIP_DURATION_MIN) AS avg_trip_duration,
    
    MIN(AVG_SPEED_MPH) AS min_avg_speed,
    MAX(AVG_SPEED_MPH) AS max_avg_speed,
    AVG(AVG_SPEED_MPH) AS avg_speed,
    
    MIN(TIP_PCT) AS min_tip_pct,
    MAX(TIP_PCT) AS max_tip_pct,
    AVG(TIP_PCT) AS avg_tip_pct

FROM base;

"""


In [5]:
df = pd.read_sql(query_val, conn)

/tmp/ipykernel_20687/1631263927.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_val, conn)


In [6]:
pd.set_option('display.max_columns', None)
df.head()

,TOTAL_REGISTROS,NULL_PICKUP_DATETIME,NULL_DROPOFF_DATETIME,NULL_PICKUP_DATE,NULL_PU_LOCATION,NULL_DO_LOCATION,NULL_TRIP_DISTANCE,NULL_TOTAL_AMOUNT,NULL_SERVICE_TYPE,NULL_PAYMENT_TYPE,DISTANCIA_FUERA_RANGO,MONTO_FUERA_RANGO,PASAJEROS_FUERA_RANGO,VELOCIDAD_FUERA_RANGO,TIP_PCT_FUERA_RANGO,DROPOFF_ANTES_PICKUP,DURACION_INVALIDA,MIN_TRIP_DISTANCE,MAX_TRIP_DISTANCE,AVG_TRIP_DISTANCE,MIN_TOTAL_AMOUNT,MAX_TOTAL_AMOUNT,AVG_TOTAL_AMOUNT,MIN_TRIP_DURATION,MAX_TRIP_DURATION,AVG_TRIP_DURATION,MIN_AVG_SPEED,MAX_AVG_SPEED,AVG_SPEED,MIN_TIP_PCT,MAX_TIP_PCT,AVG_TIP_PCT
0,800840342,0,0,0,0,0,0,0,0,0,42882,2472271,6,190827,184,84402,5830394,-40840124.4,59016609.3,5.778842,-2567.8,3950611.6,18.487614,-61001725,125373161,19.343971,-2.405684e+09,3.540926e+09,24.35337,-7933.333333,2.109288e+08,10.418488


In [7]:
query_snapshot = """
SELECT *
FROM STAGE.OBT_TRIPS
LIMIT 20
"""
df_snapshot = pd.read_sql(query_snapshot, conn)
df_snapshot.head()

/tmp/ipykernel_20687/1457297886.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_snapshot = pd.read_sql(query_snapshot, conn)


,PICKUP_DATETIME,DROPOFF_DATETIME,PICKUP_DATE,PICKUP_HOUR,DROPOFF_DATE,DROPOFF_HOUR,DAY_OF_WEEK,MONTH,YEAR,PU_LOCATION_ID,PU_ZONE,PU_BOROUGH,DO_LOCATION_ID,DO_ZONE,DO_BOROUGH,SERVICE_TYPE,VENDOR_ID,VENDOR_NAME,RATE_CODE_ID,RATE_CODE_DESC,PAYMENT_TYPE,PAYMENT_TYPE_DESC,TRIP_TYPE,TRIP_TYPE_DESC,PASSENGER_COUNT,TRIP_DISTANCE,STORE_AND_FWD_FLAG,FARE_AMOUNT,EXTRA,MTA_TAX,TIP_AMOUNT,TOLLS_AMOUNT,IMPROVEMENT_SURCHARGE,CONGESTION_SURCHARGE,AIRPORT_FEE,TOTAL_AMOUNT,TRIP_DURATION_MIN,AVG_SPEED_MPH,TIP_PCT,RUN_ID,INGESTED_AT_UTC,SOURCE_SERVICE,SOURCE_YEAR,SOURCE_MONTH
0,2016-07-27 03:53:34,2016-07-27 04:17:36,2016-07-27,3,2016-07-27,4,3,7,2016,138,LaGuardia Airport,Queens,202,Roosevelt Island,Manhattan,yellow,1,"Creative Mobile Technologies, LLC",1,Standard rate,1,Credit card,None,Unknown,1,6.30,No,21.0,0.0,0.5,4.35,0.0,0.3,None,None,26.15,24,15.750000,16.634799,run_2016_07,2025-10-20 20:35:21.429968,yellow,2016,7
1,2016-07-27 03:51:59,2016-07-27 03:56:20,2016-07-27,3,2016-07-27,3,3,7,2016,235,University Heights/Morris Heights,Bronx,235,University Heights/Morris Heights,Bronx,yellow,2,"Curb Mobility, LLC",1,Standard rate,2,Cash,None,Unknown,1,0.82,No,5.5,0.0,0.5,0.00,0.0,0.3,None,None,6.30,5,9.840039,0.000000,run_2016_07,2025-10-20 20:35:21.429968,yellow,2016,7
2,2016-07-27 03:27:11,2016-07-27 03:45:30,2016-07-27,3,2016-07-27,3,3,7,2016,170,Murray Hill,Manhattan,75,East Harlem South,Manhattan,yellow,2,"Curb Mobility, LLC",1,Standard rate,1,Credit card,None,Unknown,1,3.49,No,15.0,0.0,0.5,1.58,0.0,0.3,None,None,17.38,18,11.633333,9.090909,run_2016_07,2025-10-20 20:35:21.429968,yellow,2016,7
3,2016-07-27 03:31:30,2016-07-27 03:48:12,2016-07-27,3,2016-07-27,3,3,7,2016,263,Yorkville West,Manhattan,162,Midtown East,Manhattan,yellow,1,"Creative Mobile Technologies, LLC",1,Standard rate,1,Credit card,None,Unknown,1,2.00,No,12.0,0.0,0.5,1.66,0.0,0.3,None,None,14.46,17,7.058832,11.479945,run_2016_07,2025-10-20 20:35:21.429968,yellow,2016,7
4,2016-07-27 03:57:37,2016-07-27 04:01:18,2016-07-27,3,2016-07-27,4,3,7,2016,141,Lenox Hill West,Manhattan,141,Lenox Hill West,Manhattan,yellow,2,"Curb Mobility, LLC",1,Standard rate,1,Credit card,None,Unknown,5,0.62,No,4.5,0.0,0.5,1.06,0.0,0.3,None,None,6.36,4,9.299954,16.666667,run_2016_07,2025-10-20 20:35:21.429968,yellow,2016,7


In [8]:
conf = SparkConf()
conf.set("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.24.2,net.snowflake:spark-snowflake_2.12:3.1.2") 
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate() 

In [9]:
# Auditoría: tabla o reporte con conteos por servicio/año/mes y run_id.

sfOptions = {
    "sfURL" : "TLZAPUN-PKC06603.snowflakecomputing.com",
    "sfDatabase" : "NY_TAXI",
    "sfSchema" : "RAW",
    "sfWarehouse" :"COMPUTE_WH",
    "sfRole" : "ACCOUNTADMIN",
    "sfUser" : "MARE122510",
    "sfPassword" : "MyTurnEra2025100%"
}


df = spark.read \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "STAGE.OBT_TRIPS") \
    .load()

df_counts = df.groupBy("SOURCE_YEAR", "SOURCE_MONTH", "RUN_ID").count().orderBy("SOURCE_YEAR", "SOURCE_MONTH", "RUN_ID")
df_counts.show(truncate=False, n = 100)


+-----------+------------+-----------+--------+
|SOURCE_YEAR|SOURCE_MONTH|RUN_ID     |count   |
+-----------+------------+-----------+--------+
|2015       |1           |run_2015_01|15758021|
|2015       |2           |run_2015_02|14017224|
|2015       |3           |run_2015_03|15065525|
|2015       |4           |run_2015_04|14728152|
|2015       |5           |run_2015_05|14944525|
|2015       |6           |run_2015_06|13963804|
|2015       |7           |run_2015_07|13101337|
|2015       |8           |run_2015_08|12655466|
|2015       |9           |run_2015_09|12713049|
|2015       |10          |run_2015_10|13937869|
|2015       |11          |run_2015_11|12835224|
|2015       |12          |run_2015_12|13061293|
|2016       |1           |run_2016_01|12350359|
|2016       |2           |run_2016_02|1510722 |
|2016       |3           |run_2016_03|13780217|
|2016       |4           |run_2016_04|13471922|
|2016       |5           |run_2016_05|25201077|
|2016       |6           |run_2016_06|23